# Suitability Analysis using Arcpy.

This notebook uses arcpy to perform a suitability analysis on a DSM. First, I will extract the DSM from DRUM. Then I will merge the rasters. Finally, I will use a hillshade, reclass, and multiply the rasters. 



In [2]:
import numpy as np
import arcpy
import requests

## Part 1 Extracting the DSM From DRUM 

In [18]:
drum = "https://conservancy.umn.edu/" ##main site

handle = "handle/11299/166578/" ## Handle found in the metadata

query = "discover?query=DSM"

information =  requests.get(drum + handle + query)

new_handle = "handle/11299/172696/" #found in information.content

ashland = "q2758_DSM.zip" ##dsm needed


end = "?sequence=9&isAllowed=y" ##what I needed to add for it to work


r = requests.get(drum + "bitstream/" +new_handle + ashland + end)

open(ashland, 'wb').write(r.content)
import zipfile
with zipfile.ZipFile(ashland, 'r') as zip_ref:
    zip_ref.extractall("d:/git/GIS5572shpfiles/projdata/")

## Part 2: Merging the rasters

In [19]:
arcpy.env.workspace = "D:/git/GIS5572shpfiles/projdata/q2758/" ##Workspace where the rasters are. 

In [20]:
rasters = arcpy.ListRasters()

merge = rasters[0] ##defining the raster to merge.
rasters


['2758-01-09_2786.img',
 '2758-03-03_2652.img',
 '2758-03-07_2719.img',
 '2758-06-01_2651.img']

In [21]:
arcpy.management.Mosaic(rasters, merge) ## Merging rasters

<Result 'D:/git/GIS5572shpfiles/projdata/q2758\\2758-01-09_2786.img'>

## Part 3: Performing the Suitability Analysis. 

In [22]:
arcpy.env.workspace = 'D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb/'
arcpy.env.overwriteOutput = True  ##Default Geodatabase

In [26]:
arcpy.management.CopyRaster("D:/git/GIS5572shpfiles/projdata/q2758/2758-01-09_2786.img", 'DSM') ##merged DSM
arcpy.management.CopyRaster("D:/git/GIS5572shpfiles/projdata/q2758/2758-03-03_2652.img", "DSM_52") ##just one tile, much quicker

<Result 'D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb\\DSM_52'>

In [27]:
arcpy.ddd.HillShade("DSM_52", "HillSha_winAM",131.87, 5, "NO_SHADOWS", 1) #Input, output, azimuth, zenith, and z factor
ShadowWinteram_raster = arcpy.sa.Reclassify("HillSha_winAM", "Value", "1 255 1", "DATA"); ShadowWinteram_raster.save("D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb/WinterAM")

##Hillshade and Reclassify Winter Solstice Morning 

In [28]:
arcpy.ddd.HillShade("DSM_52", "HillSha_winPM", 228.11, 5, "NO_SHADOWS", 1)
ShadowWinterpm_raster = arcpy.sa.Reclassify("HillSha_winPM", "Value", "1 255 1", "DATA"); ShadowWinterpm_raster.save("D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb/WinterPM")

##Hillshade and Reclassify Winter Solstice Evening 

In [29]:
arcpy.ddd.HillShade("DSM_52", "HillSha_sumAM", 60.66, 5, "NO_SHADOWS", 1)
ShadowSummeram_raster = arcpy.sa.Reclassify("HillSha_sumAM", "Value", "1 255 1", "DATA"); ShadowSummeram_raster.save("D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb/SummerAM")

##Hillshade and Reclassify Summer Solstice Morning 

In [30]:
arcpy.ddd.HillShade("DSM_52", "HillSha_sumPM", 299.33, 5, "NO_SHADOWS", 1)
ShadowSummerpm_raster = arcpy.sa.Reclassify("HillSha_sumPM", "Value", "1 255 1", "DATA"); ShadowSummerpm_raster.save("D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb/SummerPM")

##Hillshade and Reclassify Winter Solstice Morning 

In [34]:
Final_52 = arcpy.Raster("WinterAM") * arcpy.Raster("WinterPM") * arcpy.Raster("SummerAM") * arcpy.Raster("SummerPM") ##Multiplying all four rasters to get one suitability output

In [36]:
Final_52.save("D:/Users/Owner/Documents/ArcGIS/Projects/Arc2_Project_4_25_2021/Arc2_Project_4_25_2021.gdb/Final") ##Saving the output. 

The final output is stored the database. This conclude the notebook.